In [1]:
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format ,from_unixtime ,monotonically_increasing_id
from pyspark.sql import types as t
from pyspark.sql import SparkSession, SQLContext, GroupedData
from pyspark.sql.functions import *

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1568404209694_0021,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .enableHiveSupport().getOrCreate()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# read  Country statstics data
df = spark.read.format("parquet").load("s3n://capstone-rk/visitors_per_country")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

233

In [4]:
df.filter(df["county"] == 'INDIA').show(300)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------+----+-----+
|county|visitors|year|month|
+------+--------+----+-----+
| INDIA|  119263|2016|    7|
| INDIA|  144313|2016|    6|
+------+--------+----+-----+

In [16]:
df.groupby("county","Year").pivot("month").avg('visitors').show(300)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+---------+---------+---------+
|              county|Year|     null|        6|        7|
+--------------------+----+---------+---------+---------+
|         SWITZERLAND|2016|     null|  35545.0|  69381.0|
|               MALTA|2016|     null|    477.0|    739.0|
|         MONTENEGRO |2016|     null|    587.0|    474.0|
|               PALAU|2016|     null|     14.0|     23.0|
|                CUBA|2016|     null|   5456.0|   6002.0|
|             NIGERIA|2016|     null|  13820.0|  25366.0|
|            DOMINICA|2016|     null|    546.0|    981.0|
|  DOMINICAN REPUBLIC|2016|     null|  36270.0|  45472.0|
|       COCOS ISLANDS|2016|     null|      3.0|      1.0|
|CENTRAL AFRICAN R...|2016|     null|     35.0|     29.0|
|          MADAGASCAR|2016|     null|    107.0|    179.0|
|               SPAIN|2016|     null|  74242.0|  84948.0|
|             LESOTHO|2016|     null|     36.0|     76.0|
|           MACEDONIA|2016|     null|    720.0|    560.0|
|             

In [5]:
df2 = df.groupby("county","Year").pivot("month").avg('visitors')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df2.printschema

In [7]:
df2.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- county: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- null: double (nullable = true)
 |-- 6: double (nullable = true)
 |-- 7: double (nullable = true)

In [61]:
#df2.select("county" , "Year" , "6","7").show()
#df3=df2.select(df2.county.alias("Country"), df2.Year.alias("Year") ,format_number(df2["6"],0).alias("June") ,format_number(df2["7"],0).alias("July")  )
df3=df2.select(df2.county.alias("Country"), df2.Year.alias("Year") , df2["6"].alias("June") , df2["7"].alias("July")  )
df3.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+-------+-------+
|             Country|Year|   June|   July|
+--------------------+----+-------+-------+
|         SWITZERLAND|2016|35545.0|69381.0|
|               MALTA|2016|  477.0|  739.0|
|         MONTENEGRO |2016|  587.0|  474.0|
|               PALAU|2016|   14.0|   23.0|
|                CUBA|2016| 5456.0| 6002.0|
|             NIGERIA|2016|13820.0|25366.0|
|            DOMINICA|2016|  546.0|  981.0|
|  DOMINICAN REPUBLIC|2016|36270.0|45472.0|
|CENTRAL AFRICAN R...|2016|   35.0|   29.0|
|       COCOS ISLANDS|2016|    3.0|    1.0|
|          MADAGASCAR|2016|  107.0|  179.0|
|               SPAIN|2016|74242.0|84948.0|
|             LESOTHO|2016|   36.0|   76.0|
|           MACEDONIA|2016|  720.0|  560.0|
|               EGYPT|2016| 7067.0|13183.0|
|              GUINEA|2016|  173.0|  368.0|
|               LIBYA|2016|   73.0|  116.0|
|                PERU|2016|22313.0|32320.0|
|              CYPRUS|2016| 1199.0| 1193.0|
|    HOLY SEE/VATICAN|2016|   12

In [86]:
df4 = df3.withColumn("% change in July" , format_number( ((( df3["July"] - df3["June"]  ) /  df3["June"] ) * 100),2))
          

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [85]:
df4.select ("Country", "year",format_number("June",0).alias("June") ,format_number("July",0).alias("July") , "% change in July"  ).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+------+------+----------------+
|             Country|year|  June|  July|% change in July|
+--------------------+----+------+------+----------------+
|         SWITZERLAND|2016|35,545|69,381|           95.19|
|               MALTA|2016|   477|   739|           54.93|
|         MONTENEGRO |2016|   587|   474|          -19.25|
|               PALAU|2016|    14|    23|           64.29|
|                CUBA|2016| 5,456| 6,002|           10.01|
|             NIGERIA|2016|13,820|25,366|           83.55|
|            DOMINICA|2016|   546|   981|           79.67|
|  DOMINICAN REPUBLIC|2016|36,270|45,472|           25.37|
|       COCOS ISLANDS|2016|     3|     1|          -66.67|
|CENTRAL AFRICAN R...|2016|    35|    29|          -17.14|
|          MADAGASCAR|2016|   107|   179|           67.29|
|               SPAIN|2016|74,242|84,948|           14.42|
|             LESOTHO|2016|    36|    76|          111.11|
|           MACEDONIA|2016|   720|   560|          -22.2